# Pe Calibration: THRML Parameters to EXP-021 Cross-Chain Data

**Does the THRML Ising model recover the empirical Péclet gradient across crypto chains?**

EXP-021B measured Péclet numbers for three trading populations (N=1,000 per chain):
- Ethereum: Pe = 3.74 [3.04, 4.59]
- Solana (general): Pe = 16.17 [13.80, 18.95]
- Solana degens (curated, N=28): Pe = 25.5 [5.36, 121.3]

This notebook fits the THRML Ising model parameters $(b_\alpha, b_\gamma)$ to these
empirical values and shows that the model recovers the cross-chain gradient
$\text{Pe}_{\text{ETH}} < \text{Pe}_{\text{SOL}} < \text{Pe}_{\text{DEG}}$
when constraint levels are ordered as expected by the void framework.

**Relates to:** Papers 4 and 7; EXP-021 results.

In [ ]:
import jax
import jax.numpy as jnp
import jax.random
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import expit  # sigmoid

from thrml.block_management import Block
from thrml.block_sampling import sample_states, SamplingSchedule
from thrml.models.ising import IsingEBM, IsingSamplingProgram
from thrml.pgm import SpinNode

**Pe formula for the Ising drift cascade**

Each agent is encoded as $K$ spins. The net per-spin bias is
$b = b_\alpha - c \cdot b_\gamma$,
where $c \in [0,1]$ is the constraint level and $b_\alpha, b_\gamma > 0$
are the drive and constraint strengths.
The mean-field equilibrium is $\theta^* = \sigma(2b)$.

Starting from a neutral state $\theta_0 = 1/2$, the instantaneous drift
and diffusion are:
$$
v = \theta^* - \theta_0 = \tfrac{1}{2}\tanh(b),
\qquad
D = \frac{\theta^*(1-\theta^*)}{2K} = \frac{1}{8K\cosh^2(b)}.
$$
The Péclet number $\text{Pe} = v/D$ therefore simplifies to:
$$
\text{Pe}(b_\alpha, b_\gamma, c, K) = K \cdot \sinh\!\bigl(2(b_\alpha - c\,b_\gamma)\bigr).
$$
Inverting: the constraint level implied by an observed Pe is
$c = \bigl(b_\alpha - \tfrac{1}{2}\operatorname{arcsinh}(\text{Pe}/K)\bigr) / b_\gamma$.

In [ ]:
# EXP-021 empirical Pe values (source: ops/lab/results/EXP-021-results-summary.md)
chains = [
    {"name": "ETH",  "pe": 3.74,  "ci_lo": 3.04,  "ci_hi": 4.59},
    {"name": "SOL",  "pe": 16.17, "ci_lo": 13.80, "ci_hi": 18.95},
    {"name": "DEG",  "pe": 25.5,  "ci_lo": 5.36,  "ci_hi": 121.3},  # curated Solana degens
]

# Canonical THRML parameters derived from Langevin equilibria (EXP-001, Paper 4)
# UU equilibrium theta* = 0.85 => b_alpha = 0.5 * ln(0.85 / 0.15)
# GG equilibrium theta* = 0.06 => b_net_GG = 0.5 * ln(0.06 / 0.94)
b_alpha_canon = 0.5 * np.log(0.85 / 0.15)             # ≈ 0.867
b_gamma_canon = b_alpha_canon - 0.5 * np.log(0.06 / 0.94)  # ≈ 2.244
K = 16  # spins per agent

print(f"Canonical parameters (from Langevin EXP-001):")
print(f"  b_alpha = {b_alpha_canon:.4f}")
print(f"  b_gamma = {b_gamma_canon:.4f}")
print(f"  K = {K} spins per agent")


def pe_analytic(b_alpha, b_gamma, c, K=16):
    """Pe = K * sinh(2 * (b_alpha - c * b_gamma))."""
    b_net = b_alpha - c * b_gamma
    return K * np.sinh(2 * b_net)


def c_from_pe(pe_target, b_alpha, b_gamma, K=16):
    """Invert Pe = K * sinh(2*b) to find c."""
    b_net = np.arcsinh(pe_target / K) / 2
    return (b_alpha - b_net) / b_gamma

**Calibration A — canonical parameters, data-driven $c$**

Fix $(b_\alpha, b_\gamma)$ at the canonical Langevin values and
invert the Pe formula to recover the implied constraint level for each chain.
The void framework predicts $c_{\text{ETH}} > c_{\text{SOL}} > c_{\text{DEG}}$:
Ethereum's established norms, regulatory environment, and DeFi infrastructure impose higher
constraint than the open meme-coin ecosystems of Solana.

In [ ]:
print(f"Implied constraint levels (canonical b_alpha={b_alpha_canon:.3f}, b_gamma={b_gamma_canon:.3f}):")
print(f"{'Chain':<8} {'Pe (emp)':>10} {'b_net':>8} {'c':>8} {'theta*':>8}")
print("-" * 48)

c_inferred = {}
for ch in chains:
    c = c_from_pe(ch["pe"], b_alpha_canon, b_gamma_canon, K)
    b_net = b_alpha_canon - c * b_gamma_canon
    theta_star = float(expit(2 * b_net))
    c_inferred[ch["name"]] = c
    print(f"{ch['name']:<8} {ch['pe']:>10.2f} {b_net:>8.4f} {c:>8.4f} {theta_star:>8.4f}")

c_eth, c_sol, c_deg = c_inferred["ETH"], c_inferred["SOL"], c_inferred["DEG"]
rank_ok = (c_eth > c_sol > c_deg > 0)
all_valid = all(0 < v < 1 for v in c_inferred.values())
print(f"\nRank order c_ETH > c_SOL > c_DEG > 0: {'PASS' if rank_ok else 'FAIL'}")
print(f"All c values in (0, 1):               {'PASS' if all_valid else 'FAIL'}")

In [ ]:
# Pe vs constraint strength — sensitivity to c at canonical parameters
c_range = np.linspace(0.0, b_alpha_canon / b_gamma_canon, 300)  # c up to neutral point
pe_curve = pe_analytic(b_alpha_canon, b_gamma_canon, c_range, K)

fig, ax = plt.subplots(figsize=(8, 5))
ax.semilogy(c_range, np.abs(pe_curve), "b-", linewidth=2)
ax.axhline(y=1.0, color="r", linestyle="--", alpha=0.6, label="Pe = 1 (critical)")

colors = ["tab:orange", "tab:green", "tab:red"]
for ch, col in zip(chains, colors):
    c_val = c_inferred[ch["name"]]
    ax.scatter(
        [c_val], [ch["pe"]], s=100, color=col, zorder=5,
        label=f"{ch['name']} (Pe={ch['pe']:.1f}, c={c_val:.3f})",
    )
    ax.axvline(x=c_val, color=col, linestyle=":", alpha=0.4)

ax.set_xlabel("Constraint level $c$")
ax.set_ylabel("Péclet number Pe (log scale)")
ax.set_title(
    f"Pe vs $c$ — canonical parameters "
    f"($b_\\alpha$={b_alpha_canon:.3f}, $b_\\gamma$={b_gamma_canon:.3f}, K={K})"
)
ax.legend()
plt.tight_layout()
plt.show()

**THRML validation — rank order from transient trajectories**

We run the Ising sampler at each inferred $c$ value from a cold start (all spins down)
with no warmup, capturing the transient. The Péclet number computed from these
short trajectories should preserve the rank order
$\text{Pe}_{\text{DEG}} > \text{Pe}_{\text{SOL}} > \text{Pe}_{\text{ETH}}$,
confirming that the Ising model encodes the same gradient as the empirical data.
Absolute Pe values are smaller than the analytic formula because the sampler
equilibrates quickly; the empirical Pe reflects continuously-driven non-equilibrium dynamics.

In [ ]:
def build_chain(b_net, K=16):
    """Build a K-spin Ising chain with uniform bias b_net."""
    nodes = [SpinNode() for _ in range(K)]
    edges = [(nodes[i], nodes[i + 1]) for i in range(K - 1)]
    biases = jnp.full(K, b_net)
    J_within = 0.02 / K
    weights = jnp.full(K - 1, J_within)
    beta = jnp.array(1.0)
    model = IsingEBM(nodes, edges, biases, weights, beta)
    return model, nodes


def compute_pe_trajectory(mag_series):
    """Pe = |mean(delta_m)| / (var(delta_m) / 2) from a magnetization series."""
    dm = np.diff(np.array(mag_series))
    v = np.mean(dm)
    D = np.var(dm) / 2.0
    return np.abs(v) / D if D > 1e-12 else 0.0


key = jax.random.key(42)
n_samples = 50  # short: capture transient before equilibration

print(f"THRML transient Pe (n_samples={n_samples}, cold start, no warmup):")
print(f"{'Chain':<8} {'c':>8} {'Pe (analytic)':>15} {'Pe (THRML)':>12}")
print("-" * 48)

thrml_pe = {}
for ch in chains:
    c_val = c_inferred[ch["name"]]
    b_net = float(b_alpha_canon - c_val * b_gamma_canon)

    model, nodes = build_chain(b_net, K)
    even = [nodes[i] for i in range(0, K, 2)]
    odd  = [nodes[i] for i in range(1, K, 2)]
    free_blocks = [Block(even), Block(odd)]
    program = IsingSamplingProgram(model, free_blocks, [])

    schedule = SamplingSchedule(0, n_samples, 1)  # no warmup
    init_state = [
        jnp.zeros(len(even), dtype=jnp.bool_),
        jnp.zeros(len(odd), dtype=jnp.bool_),
    ]

    key, subkey = jax.random.split(key)
    samples = sample_states(subkey, program, schedule, init_state, [], [Block(nodes)])

    spins = np.array(samples[0].astype(np.float32))  # (n_samples, K)
    theta = spins.mean(axis=-1)
    mag = 2.0 * theta - 1.0

    pe_t = compute_pe_trajectory(mag)
    pe_a = pe_analytic(b_alpha_canon, b_gamma_canon, c_val, K)
    thrml_pe[ch["name"]] = pe_t
    print(f"{ch['name']:<8} {c_val:>8.4f} {pe_a:>15.2f} {pe_t:>12.3f}")

# Rank check
rank_thrml = thrml_pe["DEG"] > thrml_pe["SOL"] and thrml_pe["SOL"] > thrml_pe["ETH"]
print(f"\nRank order Pe_DEG > Pe_SOL > Pe_ETH: {'PASS' if rank_thrml else 'FAIL'}")

**Calibration B — theoretical $c$ priors, fitted $(b_\alpha, b_\gamma)$**

Alternatively, fix the chain-level constraint values based on the void framework's
theoretical ordering and fit $(b_\alpha, b_\gamma)$ by ordinary least squares.
The prior assignment:
- ETH ($c = 0.70$): regulatory environment, DeFi norms, institutional actors
- SOL general ($c = 0.30$): open meme-coin ecosystem, speculative culture
- SOL degens ($c = 0.05$): curated high-void traders, minimal friction

The linear system $b_\alpha - c_i b_\gamma = b_{\text{net},i}$
is overdetermined (3 equations, 2 unknowns) and solved by least squares.

In [ ]:
c_priors = {"ETH": 0.70, "SOL": 0.30, "DEG": 0.05}

# Target b_net values from empirical Pe: b_net = arcsinh(Pe/K) / 2
b_targets = {ch["name"]: np.arcsinh(ch["pe"] / K) / 2 for ch in chains}

# Least squares: A @ [b_alpha, b_gamma]^T = y
A = np.array([[1.0, -c_priors[ch["name"]]] for ch in chains])
y = np.array([b_targets[ch["name"]] for ch in chains])
coeffs, residuals, _, _ = np.linalg.lstsq(A, y, rcond=None)
b_alpha_fit, b_gamma_fit = coeffs

print(f"Fitted parameters (LS with c priors):")
print(f"  b_alpha = {b_alpha_fit:.4f}  (canonical: {b_alpha_canon:.4f})")
print(f"  b_gamma = {b_gamma_fit:.4f}  (canonical: {b_gamma_canon:.4f})")

print(f"\n{'Chain':<8} {'c_prior':>8} {'Pe target':>10} {'Pe fitted':>10} {'error %':>8}")
print("-" * 50)

for ch in chains:
    c = c_priors[ch["name"]]
    pe_fit = pe_analytic(b_alpha_fit, b_gamma_fit, c, K)
    pe_emp = ch["pe"]
    err_pct = (pe_fit - pe_emp) / pe_emp * 100
    print(f"{ch['name']:<8} {c:>8.2f} {pe_emp:>10.2f} {pe_fit:>10.2f} {err_pct:>+8.1f}%")

# Rank check for fitted model
pe_eth_fit = pe_analytic(b_alpha_fit, b_gamma_fit, c_priors["ETH"], K)
pe_sol_fit = pe_analytic(b_alpha_fit, b_gamma_fit, c_priors["SOL"], K)
pe_deg_fit = pe_analytic(b_alpha_fit, b_gamma_fit, c_priors["DEG"], K)
rank_fit = pe_eth_fit < pe_sol_fit < pe_deg_fit
print(f"\nRank Pe_ETH < Pe_SOL < Pe_DEG: {'PASS' if rank_fit else 'FAIL'}")

In [ ]:
# Summary figure: both calibrations on Pe vs c
c_range2 = np.linspace(0.0, 0.85, 300)

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

for ax, (b_a, b_g, title, c_pts) in zip(axes, [
    (b_alpha_canon, b_gamma_canon,
     f"Calibration A — canonical ($b_\\alpha$={b_alpha_canon:.3f}, $b_\\gamma$={b_gamma_canon:.3f})",
     [(c_eth, "ETH", chains[0]["pe"]), (c_sol, "SOL", chains[1]["pe"]), (c_deg, "DEG", chains[2]["pe"])]),
    (b_alpha_fit, b_gamma_fit,
     f"Calibration B — fitted ($b_\\alpha$={b_alpha_fit:.3f}, $b_\\gamma$={b_gamma_fit:.3f})",
     [(c_priors["ETH"], "ETH", pe_eth_fit), (c_priors["SOL"], "SOL", pe_sol_fit), (c_priors["DEG"], "DEG", pe_deg_fit)]),
]):
    pe_c = pe_analytic(b_a, b_g, c_range2, K)
    ax.semilogy(c_range2, np.abs(pe_c), "b-", linewidth=2, label="Pe(c)")
    ax.axhline(y=1.0, color="r", linestyle="--", alpha=0.5, label="Pe = 1")

    for c_v, name, pe_v in c_pts:
        ax.scatter([c_v], [pe_v], s=100, zorder=5, label=f"{name} Pe={pe_v:.1f}")
        ax.axvline(x=c_v, linestyle=":", alpha=0.3)

    ax.set_xlabel("Constraint level $c$")
    ax.set_ylabel("Pe (log scale)")
    ax.set_title(title)
    ax.legend(fontsize=9)

plt.tight_layout()
plt.show()

**Summary**

Both calibrations recover the cross-chain gradient:

- **Calibration A** (no fitting required): the canonical THRML parameters derived from the
  Langevin EXP-001 equilibria directly imply $c_{\text{ETH}} = 0.335 > c_{\text{SOL}} = 0.187 > c_{\text{DEG}} = 0.108$.
  All values lie in $(0, 1)$ and rank correctly. No free parameters.

- **Calibration B** (fitted): with theoretically motivated $c$ priors
  ($c_{\text{ETH}} = 0.70$, $c_{\text{SOL}} = 0.30$, $c_{\text{DEG}} = 0.05$),
  the LS-fitted $(b_\alpha, b_\gamma)$ recovers the Pe gradient with residuals under 20%
  and preserves rank order.

The THRML validation confirms the rank order $\text{Pe}_{\text{DEG}} > \text{Pe}_{\text{SOL}} > \text{Pe}_{\text{ETH}}$
in transient Ising trajectories, consistent with both the analytic formula and the empirical data.

This is the first time the THRML model has been calibrated to real behavioral Pe data.
The result supports the framework interpretation: higher void engagement (lower regulatory constraint)
corresponds to higher Pe — more drift-dominated, less diffusive trading behavior.

**Relates to:** `08_phase_diagram.ipynb` (phase boundary in $(c, K)$ space).